# Simplified survival analysis workflow for the auwerxlab infrastructure

Requires RENKU and the [SLIMS](https://www.genohm.com) laboratory information management system.

#### 1. Setup


In [ ]:
# General setup
%run lib/workflow
import os
wf = Workflow(url = input("SLIMS REST URL (ex: https://slims-lisp.epfl.ch/rest/rest): "),
              proj = input("SLIMS project name (if any): "),
              exp = input("SLIMS experiment name: "),
              user = input("SLIMS user name: "))

input_dataset_name = "counts"
results_dataset_name = "results"
input_dir = os.path.join("data", input_dataset_name)
results_dir =  os.path.join("data/", results_dataset_name)

#### 2. Download counts data from a SLIMS ELN attachment step

In [ ]:
# Download counts data from a SLIMS ELN attachment step
downloaded_file = wf.slims_fetch(output_dir = input_dir)

#### 3. Create a RENKU dataset for the counts data

In [ ]:
# Create a RENKU dataset for the counts data
!sed -i "/\/data\/\*/d;/\/figs\/\*/d;/\*\.nb\.html/d" .gitignore
input_dataset_metadata = wf.dataset_create(dataset_name = input_dataset_name)
wf.dataset_add(dataset_name = input_dataset_name,
              file = downloaded_file)

#### 4. Analyze the counts data

In [ ]:
# Use the docker image packrat directory
!renku-r ln-packrat-lib -p . -s /home/rstudio/packrat -v -f
# Analyze the counts data using the lib/build_survival_curves.R script
wf.build_survival_curves(input_fp = downloaded_file,
                         output_dir = results_dir)

#### 5. Create a RENKU dataset for the analysis results

In [ ]:
# Create a RENKU dataset for the analysis results
!sed -i "/\/data\/\*/d;/\/figs\/\*/d;/\*\.nb\.html/d" .gitignore
results_dataset_metadata = wf.dataset_create(dataset_name = results_dataset_name)
wf.dataset_add(dataset_name = results_dataset_name,
               file = results_dir)

#### 6. Upload the results into SLIMS

In [ ]:
# Upload the results into SLIMS
wf.slims_add_dataset(
    files = ",".join(
        [os.path.join(results_dir, e)
         for e in os.listdir(results_dir)
         if os.path.isfile(os.path.join(results_dir, e))
        ]) + "," + results_dataset_metadata
)

#### 7. Save your work in RENKU!

In [ ]:
%%bash -s "{input('Git commit message: ')}"
sed -i "/\/data\/\*/d;/\/figs\/\*/d;/\*\.nb\.html/d" .gitignore
git add -A
git commit -m "$1"
git push